In [26]:
import numpy as np
import pandas as pd
import pickle
import gpflow
from scipy.optimize import minimize
from scipy.optimize import differential_evolution

In [27]:
import sys
sys.path.append('/glade/u/home/linnia/ctsm6_ppe/')
from utils.pyfunctions import *
utils_path = '/glade/u/home/linnia/ctsm6_ppe/utils/'

In [28]:
def get_biome_param_names(b):
    u_params = ['jmaxb1', 'wc2wjb0', 'grperc', 'theta_cj', 'tpu25ratio', 'lmrse',
       'vcmaxha', 'jmaxha', 'tpuha', 'lmrha', 'ACCLIM_SF', 'fstor2tran',
       'crit_onset_gdd_sf', 'FUN_fracfixers', 'fff', 'e_ice', 'bsw_sf',
       'sucsat_sf', 'watsat_sf', 'hksat_sf', 'om_frac_sf', 'baseflow_scalar',
       'maximum_leaf_wetted_fraction', 'interception_fraction', 'cv', 'd_max',
       'z0v_Cr', 'n_melt_coef', 'accum_factor', 'xdrdt',
       'upplim_destruct_metamorph', 'snw_rds_refrz', 'decomp_depth_efolding',
       'TAU', 'q10_mr', 'minpsi_hr', 'maxpsi_hr', 'RF_LS', 'RF_SS', 'RF_CWD',
       'pot_hmn_ign_counts_alpha']

    pft_params   = ['kmax','psi50','jmaxb0','slatop','lmr_intercept_atkin',
                'medlynslope','medlynintercept','froot_leaf','leafcn','leaf_long',
               'KCN','dleaf','r_mort','fsr_pft','xl']
    
    with open(utils_dir+"/biome_configs.pkl", "rb") as f:
        biome_configs = pickle.load(f)

    param_names = list(u_params)
    for pft in biome_configs[b]['pfts']:
        pft_param_names = [f"{param}_{pft}" for param in pft_params]
        param_names.extend(pft_param_names)

    return param_names

In [29]:
# get parameter info
key = '/glade/work/linnia/CLM6-PPE/ctsm6_lhc/ctsm6lhc_11262024.txt'
params_lhc = pd.read_csv(key).drop(columns='member')

pft_params   = ['kmax','psi50','jmaxb0','slatop','lmr_intercept_atkin',
                'medlynslope','medlynintercept','froot_leaf','leafcn','leaf_long',
               'KCN','dleaf','r_mort','fsr_pft','xl']
pftix=np.array([p in pft_params for p in params_lhc.columns])
u_params = params_lhc.columns[~pftix]

In [30]:
# get biome information
with open(utils_dir+"/biome_configs.pkl", "rb") as f:
    biome_configs = pickle.load(f)

# get observations
obs_biome = xr.open_dataset(utils_path + 'wave2_obsStatistics_sudokuBiomes.nc')

# get default parameter set
default_params = pd.read_csv('default_params_norm.csv', index_col=False)

# reset some settings of default parameters
default_params.loc[0, ['jmaxb1']] = [0.4]
default_params.loc[0, ['theta_cj']] = [0.7]
default_params.loc[0, ['upplim_destruct_metamorph']] = [1]

In [31]:
# get observations & set stdev so high the optimization ignores some biome/variable combinations
obs = xr.open_dataset(utils_path + 'wave2_obsStatistics_sudokuBiomes.nc')
obs_biome = obs.copy()
obs_biome['biomassC_stdev'].loc[{'biome': 12}] = 100

### Scipy minimize optimization functions

In [32]:
def misfit(x, universal_set, biome_info, epsilon=None):

    total_error = 0

    for biome in biome_info:
        full_sample = np.concatenate([universal_set, x])
        ix = biome['param_indices']
        biome_sample = full_sample[ix]

        for emulator, target, stdev in zip(biome['emulators'], biome['targets'], biome['stdevs']):
            y_pred, _ = emulator.predict(biome_sample.reshape(1,-1))

            z = np.abs((y_pred.numpy().ravel() - target) / stdev)

            if epsilon is not None:
                # Mask out errors within threshold
                mask = z > epsilon
                # Sum squared errors only for samples exceeding threshold
                total_error += np.sum(z[mask]**2)
            else:
                total_error += np.sum(z**2)

    return total_error

In [37]:
# Wrapper to run optimization for multiple biomes
def run_optimization(x0, biome_info, method='L-BFGS-B', tol=1e-3, maxiter=5000, epsilon=None):
    
    bounds = [(0, 1)] * len(x0)

    result = minimize(
        misfit,
        x0,
        args=(universal_set, biome_info, epsilon),
        bounds=bounds,
        method=method,
        options={'ftol': tol, 'maxiter': maxiter, 'disp':True}
    )
    return result

In [93]:
# Wrapper to run optimization for multiple biomes
def run_optimization_batch(x0, biome_info, method='L-BFGS-B', tol=1e-3, maxiter=5000, epsilon=None):
    
    bounds = [(0, 1)] * len(x0)

    result = differential_evolution(
        func=misfit,
        bounds=bounds,
        args=(universal_set, biome_info, epsilon),
        strategy='best1bin',
        popsize=100,
        init=x0,
        maxiter=maxiter,
        tol=tol,
        disp=True,          # print progress each generation
        vectorized=True     # key: pass the entire population at once
    )
    return result

### Calibrate

In [34]:
# setup
paths = {
    'lai': '/glade/u/home/linnia/ctsm6_ppe/calibration/emulators_biomelai/',
    'gpp': '/glade/u/home/linnia/ctsm6_ppe/calibration/emulators_biomegpp/',
    'biomass': '/glade/u/home/linnia/ctsm6_ppe/calibration/emulators_biomebiomass/'
}

#biomes = [1,2,3,5,6,7,8,9,10,11,12,13]
biomes = [1]
biome_info = []
for b in biomes:
    biome_name = biome_configs[b]['name']
    param_names = get_biome_param_names(b)
    param_indices = [default_params.columns.get_loc(p)for p in param_names]

    emulators = [
        tf.saved_model.load(f"{paths['lai']}{biome_name}"),
        tf.saved_model.load(f"{paths['gpp']}{biome_name}"),
        tf.saved_model.load(f"{paths['biomass']}{biome_name}")
    ]
    targets = [
        obs_biome.LAI_mean.isel(biome=b).values,
        obs_biome.GPP_mean.isel(biome=b).values,
        obs_biome.biomassC_mean.isel(biome=b).values
    ]
    target_stdevs = [
        obs_biome.LAI_stdev.isel(biome=b).values,
        obs_biome.GPP_stdev.isel(biome=b).values,
        obs_biome.biomassC_stdev.isel(biome=b).values
    ]

    biome_info.append({
        'biome': b,
        'param_indices': param_indices,
        'emulators': emulators,
        'targets': targets,
        'stdevs': target_stdevs,
    })


2025-04-25 10:08:46.494137: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [35]:
# initialization
x0 = np.random.rand(195)

In [38]:
%%time
# calibration with scipy.minimize for all biomes simultaneously (195 PFT parameters: universal parameters fixed)
tol = 1E-2
maxiter=100
epsilon=3 # early stopping z-score threshold

universal_set = default_params[u_params].iloc[[0]].values[0]

# Run optimization
result = run_optimization(x0, biome_info, tol=tol, maxiter=maxiter, epsilon=epsilon)


CPU times: user 6min 38s, sys: 13.2 s, total: 6min 51s
Wall time: 25min 19s


In [37]:
# one biome (1: 56 params) and one sample with a tolerance of 3 z-scores took 54 minutes. 

(195,)